#### Pip installs

pip install python-dotenv 
<br>
pip install neo4j
<br>
pip install redis

In [24]:
import neo4j as neo4j
from neo4j import GraphDatabase
import dotenv as dotenv
from dotenv import load_dotenv
import os
import redis
import random

In [25]:
# Neo4j Connection
load_dotenv()

URI = "neo4j://localhost:7687"
AUTH = ("neo4j", os.getenv("NEO4J_PASSWORD"))

driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()

## Part 4 - Webpage Uploading

In [26]:
#store nodes and edges
nodes = {}
edges = []

with open("./data/Hollins data", "r") as file:
    for line in file:

        #seperate the two parts of the node/edge
        parts = line.strip().split()

        #exclude blank lines
        if len(parts) == 2:

            #node
            if ("http" in parts[1]):
                nodes[parts[0]] = parts[1]
            #edge
            else:
                edges.append((parts[0], parts[1]))

#gut check
print(nodes['1'])
print(edges[0])

http://www1.hollins.edu/
('6012', '23875')


In [27]:
### VERIFY THAT NEO4J HAS NO NODES AND THEN ONLY RUN ONCE ###
# import nodes to Neo4j
for page_id, page_url in nodes.items():
  
    #create a new webpage with the given id and url
    driver.execute_query(
        "CREATE (:Webpage {id: $id, url: $url})",
        id = page_id,
        url = page_url,
    )

In [28]:
### VERIFY THAT NEO4J HAS NO EDGES AND THEN ONLY RUN ONCE ###
# import edges to Neo4j
for first_page, second_page in edges:
    
    #create a new linkage between page1 and page2
    driver.execute_query(
        "MATCH (a:Webpage {id: $source}), (b:Webpage {id: $target}) MERGE (a)-[:LINKS_TO]->(b)", 
        source=first_page, 
        target=second_page,
    )

# Part 5 - Graph Characteristics

In [50]:
# Project a graph and name it myGraph
driver.execute_query(
    """ MATCH (source:Webpage)-[r:LINKS_TO]->(target:Webpage) 
    RETURN gds.graph.project('cypherGraph',source,target)"""
)

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke function `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: Graph cypherGraph already exists}

In [29]:
# How many Webpages are there?

num_nodes = driver.execute_query(
    "MATCH (n:Webpage) RETURN COUNT(n) AS NumberOfNodes"
)

print("The Webpage graph has " + str(num_nodes.records[0]["NumberOfNodes"]) + " nodes.")

The Webpage graph has 6012 nodes.


In [30]:
# How many links are found?

num_edges = driver.execute_query(
    "MATCH ()-[r:LINKS_TO]->() RETURN COUNT(r) AS NumberOfEdges"
)

print("The Webpage graph has " + str(num_edges.records[0]["NumberOfEdges"]) + " edges.")

The Webpage graph has 23875 edges.


In [ ]:
# Longest Shortest Path
# What two nodes are furthest apart via their shortest path?

longest_shortest_path = driver.execute_query(
    """
    CALL gds.allShortestPaths.stream('cypherGraph')
    YIELD sourceNodeId, targetNodeId, distance
    WITH sourceNodeId, targetNodeId, distance
    WHERE gds.util.isFinite(distance) = true
    WITH gds.util.asNode(sourceNodeId) AS source, gds.util.asNode(targetNodeId) AS target, distance WHERE source <> target

    RETURN source.url AS source, target.url AS target, distance
    ORDER BY distance DESC, source ASC, target ASC
    LIMIT 10
    """
)

print("Top 10 longest shortest paths")
for i in range(10):
    print(longest_shortest_path.records[i])


Top 10 longest shortest paths
<Record source='http://www1.hollins.edu/classes/dance/website/nancyweb/performance.html' target='http://www1.hollins.edu/Docs/Academics/divisioni/classical%20studies/saloweyca/clas%20245/arapacis1/img001.htm' distance=23.0>
<Record source='http://www1.hollins.edu/classes/dance/website/nancyweb/performance.html' target='http://www1.hollins.edu/Docs/Academics/divisioni/classical%20studies/saloweyca/clas%20245/arapacis1/navbtn.htm' distance=23.0>
<Record source='http://www1.hollins.edu/classes/dance/website/nancyweb/performance.html' target='http://www1.hollins.edu/Docs/Academics/divisioni/classical%20studies/saloweyca/clas%20245/arapacis1/note001.htm' distance=23.0>
<Record source='http://www1.hollins.edu/classes/dance/website/nancyweb/performance.html' target='http://www1.hollins.edu/Docs/Academics/divisioni/classical%20studies/saloweyca/clas%20245/arapacis1/outlinec.htm' distance=23.0>
<Record source='http://www1.hollins.edu/classes/dance/website/nancyweb/

In [33]:
# Which nodes have the most incoming and outgoing links?

largest_degrees_outgoing = driver.execute_query(
    """
    MATCH (n:Webpage)
    RETURN n, COUNT {(n)-[:LINKS_TO]->()} AS Degree
    ORDER BY Degree DESC
    LIMIT 3
    """
)

largest_degrees_incoming = driver.execute_query(
    """
    MATCH (n:Webpage)
    RETURN n, COUNT {(n)<-[:LINKS_TO]-()} AS Degree
    ORDER BY Degree DESC
    LIMIT 3
    """
)

print("The top 3 nodes with the highest number of outgoing links:")
for record in largest_degrees_outgoing.records:
    print(str(record["n"]["url"]) + " with a degree of " + str(record["Degree"]))

print("-")

print("The top 3 nodes with the highest number of incoming links:")
for record in largest_degrees_incoming.records:
    print(str(record["n"]["url"]) + " with a degree of " + str(record["Degree"]))# Consider Centrality and Community Detection



The top 3 nodes with the highest number of outgoing links:
http://www1.hollins.edu/Docs/GVCalendar/gvarchives.htm with a degree of 184
http://www1.hollins.edu/docs/gvcalendar/gvarchives.htm with a degree of 184
http://www.hollins.edu/sitemap/sitemap.htm with a degree of 177
-
The top 3 nodes with the highest number of incoming links:
http://www.hollins.edu/ with a degree of 829
http://www.hollins.edu/admissions/visit/visit.htm with a degree of 454
http://www.hollins.edu/about/about_tour.htm with a degree of 435


In [34]:
# What is the average amount of incoming and outgoing links on a Webpage?

average_degrees_outgoing = driver.execute_query(
    """
    MATCH (n:Webpage)
    WITH n, COUNT {(n)-[:LINKS_TO]->()} AS outgoingCount
    RETURN AVG(outgoingCount) AS AvgOutgoingDegree
    """
)

average_degrees_incoming = driver.execute_query(
    """
    MATCH (n:Webpage)
    WITH n, COUNT {(n)<-[:LINKS_TO]-()} AS incomingCount
    RETURN AVG(incomingCount) AS AvgIncomingDegree
    """
)

print("The average number of links on a Webpage is " + str(average_degrees_outgoing.records[0]["AvgOutgoingDegree"]))
print("The average number of links to a Webpage is " + str(average_degrees_incoming.records[0]["AvgIncomingDegree"]))

The average number of links on a Webpage is 3.9712242182302
The average number of links to a Webpage is 3.9712242182301867


In [35]:
# Centrality: Betweenness Centrality
# Which Webpages appear the most on the shortest path between 2 Webpages?

betweenness_result = driver.execute_query(
    """
    CALL gds.betweenness.stream('cypherGraph')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).url AS url, score
    ORDER BY score DESC
    LIMIT 3
    """
)

print("Top 3 Webpages with the highest Betweenness Centrality scores (pages found most often in shortest paths):")
for record in betweenness_result.records:
    print(f"URL: {record['url']} has a score of {record['score']}")



Top 3 Webpages with the highest Betweenness Centrality scores (pages found most often in shortest paths):
URL: http://www.hollins.edu/ has a score of 4384353.2777416175
URL: http://www.hollins.edu/academics/comptech/infotech.htm has a score of 2614679.6005932465
URL: http://www1.hollins.edu/docs/comptech/mainemail.htm has a score of 2612601.3841600944


In [36]:
# Community Detection: Label Propagation
# Which communities of Webpages can be found based on connections to other Webpages alone?

label_prop = driver.execute_query(
    """
    CALL gds.labelPropagation.stream('cypherGraph')
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).url AS Name, Community
    ORDER BY Community, Name
    LIMIT 20
    """
)

print("Small sample of detected Webpage communities:")
for record in label_prop.records:
    print(f"{record['Name']} is in community {record['Community']}")


Small sample of detected Webpage communities:
http://www1.hollins.edu/Docs/CompTech/Network/webmail_faq.htm is in community 2
http://www1.hollins.edu/Docs/CampusLife/StudentAct/Calendar.htm is in community 17
http://www1.hollins.edu/Docs/CampusLife/StudentAct/Contacts.htm is in community 17
http://www1.hollins.edu/Docs/CampusLife/StudentAct/Index.htm is in community 17
http://www1.hollins.edu/Docs/CampusLife/StudentAct/Student%20Activities%20News.htm is in community 17
http://www1.hollins.edu/Docs/CampusLife/StudentAct/Student%20Activities_files/Activities%20Pictures/pic.%204.htm is in community 17
http://www1.hollins.edu/Docs/CampusLife/StudentAct/The%20Faces%20of%20Student%20Activities.htm is in community 17
http://www1.hollins.edu/Docs/CampusLife/StudentAct/studentorgs/SGA/SGA%20ByLaws%202003.htm is in community 17
http://www1.hollins.edu/Docs/CampusLife/StudentAct/studentorgs/SGA/constitution.htm is in community 17
http://www1.hollins.edu/Docs/CampusLife/StudentAct/studentorgs/SGA/

# Part 6 - PageRank Algorithm

In [37]:
# Estimate the memory requirements for running the algorithm
driver.execute_query(
    """CALL gds.pageRank.write.estimate('cypherGraph', {
        writeProperty: 'pageRank',
        maxIterations: 20,
        dampingFactor: 0.85
      })
      YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory""", 
        
    )

EagerResult(records=[<Record nodeCount=6012 relationshipCount=23875 bytesMin=145864 bytesMax=145864 requiredMemory='142 KiB'>], summary=<neo4j._work.summary.ResultSummary object at 0x1192d8940>, keys=['nodeCount', 'relationshipCount', 'bytesMin', 'bytesMax', 'requiredMemory'])

In [38]:
# Run the PageRank algorithm in stream mode
driver.execute_query(
    """ CALL gds.pageRank.stream('cypherGraph')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).url AS name, score
    ORDER BY score DESC, name ASC
    """
)


EagerResult(records=[<Record name='http://www.hollins.edu/' score=50.96321589429354>, <Record name='http://www.hollins.edu/admissions/visit/visit.htm' score=23.670274055804544>, <Record name='http://www.hollins.edu/about/about_tour.htm' score=21.94585507202142>, <Record name='http://www.hollins.edu/htdig/index.html' score=20.54116778982502>, <Record name='http://www.hollins.edu/admissions/info-request/info-request.cfm' score=20.456286702214165>, <Record name='http://www.hollins.edu/admissions/apply/apply.htm' score=18.26343859539373>, <Record name='http://www.hollins.edu/academics/library/resources/web_linx.htm' score=16.963507839488624>, <Record name='http://www.hollins.edu/admissions/admissions.htm' score=15.241845750629771>, <Record name='http://www.hollins.edu/academics/academics.htm' score=14.219213812046178>, <Record name='http://www.hollins.edu/grad/coedgrad.htm' score=11.149806536086036>, <Record name='http://www1.hollins.edu/faculty/saloweyca/clas%20395/Sculpture/sld001.htm' s

In [39]:
# Changing number of iterations x=5
results_5 = driver.execute_query(
    """ CALL gds.pageRank.stream('cypherGraph', {maxIterations:5})
    YIELD nodeId, score
    MATCH (node) WHERE id(node) = nodeId
    RETURN gds.util.asNode(nodeId).url AS name, score
    ORDER BY score DESC, name ASC
    """
)

# Store the page rank with 5 iterations
ranking_5 = []
ranking_5_dict = {}
for page in results_5.records:
    ranking_5.append(page["name"])
    ranking_5_dict[page["name"]] = page["score"]

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 24, offset: 107} for query: " CALL gds.pageRank.stream('cypherGraph', {maxIterations:5})\n    YIELD nodeId, score\n    MATCH (node) WHERE id(node) = nodeId\n    RETURN gds.util.asNode(nodeId).url AS name, score\n    ORDER BY score DESC, name ASC\n    "


In [40]:
# Changing number of iterations maxIterations:15
results_15 = driver.execute_query(
    """ CALL gds.pageRank.stream('cypherGraph', {maxIterations:15})
    YIELD nodeId, score
    MATCH (node) WHERE id(node) = nodeId
    RETURN gds.util.asNode(nodeId).url AS name, score
    ORDER BY score DESC, name ASC
    """
)

# Store the page rank with 15 iterations
ranking_15 = []
ranking_15_dict = {}
for page in results_15.records:
    ranking_15.append(page["name"])
    ranking_15_dict[page["name"]] = page["score"]

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 24, offset: 108} for query: " CALL gds.pageRank.stream('cypherGraph', {maxIterations:15})\n    YIELD nodeId, score\n    MATCH (node) WHERE id(node) = nodeId\n    RETURN gds.util.asNode(nodeId).url AS name, score\n    ORDER BY score DESC, name ASC\n    "


In [41]:
# Changing number of iterations maxIterations:25
results_25 = driver.execute_query(
    """ CALL gds.pageRank.stream('cypherGraph', {maxIterations:25})
    YIELD nodeId, score
    MATCH (node) WHERE id(node) = nodeId
    RETURN gds.util.asNode(nodeId).url AS name, score
    ORDER BY score DESC, name ASC
    """
)

# Store the page rank with 25 iterations
ranking_25 = []
ranking_25_dict = {}
for page in results_25.records:
    ranking_25.append(page["name"])
    ranking_25_dict[page["name"]] = page["score"]

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 24, offset: 108} for query: " CALL gds.pageRank.stream('cypherGraph', {maxIterations:25})\n    YIELD nodeId, score\n    MATCH (node) WHERE id(node) = nodeId\n    RETURN gds.util.asNode(nodeId).url AS name, score\n    ORDER BY score DESC, name ASC\n    "


In [42]:
# PageRank Analysis: How does number of iterations alter the rank?

# How many pages changed rank in 15 iterations and 25 iterations?
rank_dict1 = {value: index for index, value in enumerate(ranking_5)}
# check how many of the initial webpages are still ranked in the same spot after increasing iterations
changed_ranks_15 = sum(1 for value in ranking_15 if rank_dict1[value] != ranking_15.index(value))
changed_ranks_25 = sum(1 for value in ranking_25 if rank_dict1[value] != ranking_25.index(value))

print(str(changed_ranks_15) + " web pages changed rank after adding 10 iterations.")
print(str(changed_ranks_25) + " web pages changed rank after adding 20 iterations.")

# How many changed between 15 and 25 iterations?
rank_dict2 = {value: index for index, value in enumerate(ranking_15)}
changed_ranks_25 = sum(1 for value in ranking_25 if rank_dict2[value] != ranking_25.index(value))
print("Only " + str(changed_ranks_25) + " web pages changed rank between 15 and 25 iterations.")



5638 web pages changed rank after adding 10 iterations.
5647 web pages changed rank after adding 20 iterations.
Only 3676 web pages changed rank between 15 and 25 iterations.


# Part 7 - Redis Uploading

In [43]:
redis_client = redis.Redis(host='localhost', port=9876, db=0)

# For every webpage
for url in nodes.values():

    # Store each of the 3 page rank versions
    page_ranks = []
    page_ranks.append(ranking_5_dict[url])
    page_ranks.append(ranking_15_dict[url])
    page_ranks.append(ranking_25_dict[url])

    # Store the value in redis
    # Key=url, value=[rank with 5 iterations, 15 iterations, 25 iterations]
    redis_client.set(url, str(page_ranks))



In [44]:
# Gut check
get_score = redis_client.get("http://www.hollins.edu/")
retrieved_value = eval(get_score.decode("utf-8"))
print(retrieved_value)

[36.16495529895856, 50.01699770424452, 51.24219788652768]


In [45]:
#quick helper to get page rank for a given url
def get_pagerank(url):
    ranks = redis_client.get(url)
    ranks = eval(ranks.decode("utf-8"))
    return ranks

# Part 8 - Sampling

In [46]:
# Random sampling! 
all_urls = list(nodes.values())

# 10 subsets of size 10
subsets_of_size_10 = [random.sample(all_urls, 10) for i in range(10)]

# 10 subsets of size 50
subsets_of_size_50 = [random.sample(all_urls, 50) for i in range(10)]

In [47]:
# Subset rankings for 5 iterations of page rank

# Sort all the subsets by their page rank
sorted_10 = [sorted(set, key=lambda url: get_pagerank(url)[0], reverse=True) for set in subsets_of_size_10]
sorted_50 = [sorted(set, key=lambda url: get_pagerank(url)[0], reverse=True) for set in subsets_of_size_50]

print("Size 10 subsets for 5 iterations of PageRank")
for set in sorted_10: 
    print([(url, get_pagerank(url)[0]) for url in set])

print("Size 50 subsets for 5 iterations of PageRank")
for set in sorted_50: 
    print([(url, get_pagerank(url)[0]) for url in set])

Size 10 subsets for 5 iterations of PageRank
[('http://www.hollins.edu/academics/library/information/gov_tax.htm', 0.5349804033627719), ('http://www1.hollins.edu/docs/alumdev/confidinfovolun.htm', 0.40617213853211287), ('http://www1.hollins.edu/faculty/saloweyca/clas%20395/pots/sld013.htm', 0.37134679440758667), ('http://www1.hollins.edu/faculty/saloweyca/clas%20395/Acropolis/tsld010.htm', 0.32906027882160344), ('http://www1.hollins.edu/faculty/clarkjm/Stat140/140pic3.jpg', 0.17232154837553093), ('http://www1.hollins.edu/docs/academics/divisioni/classical%20studies/saloweyca/clas%20245/arapacis/apa.julioclaudian3.jpg', 0.1613409502777778), ('http://www1.hollins.edu/Registrar/Forms/Intermaj.pdf', 0.15427816262168037), ('http://www1.hollins.edu/docs/Career_Development/majors/links/child.htm', 0.15278840190699544), ('http://www1.hollins.edu/registrar/MAJORS-MINORS/MAJ-MIN%2099-00/COMPUTATIONALSCIENCES%20MATH-STAT%20CONC.pdf', 0.15252582573551626), ('http://www1.hollins.edu/homepages/salow

In [48]:
# Subset rankings for 15 iterations of page rank

# Sort all the subsets by their page rank
sorted_10 = [sorted(set, key=lambda url: get_pagerank(url)[1], reverse=True) for set in subsets_of_size_10]
sorted_50 = [sorted(set, key=lambda url: get_pagerank(url)[1], reverse=True) for set in subsets_of_size_50]

print("Size 10 subsets for 15 iterations of PageRank")
for set in sorted_10: 
    print([(url, get_pagerank(url)[1]) for url in set])

print("Size 50 subsets for 15 iterations of PageRank")
for set in sorted_50: 
    print([(url, get_pagerank(url)[1]) for url in set])

Size 10 subsets for 15 iterations of PageRank
[('http://www.hollins.edu/academics/library/information/gov_tax.htm', 0.5367767138582487), ('http://www1.hollins.edu/faculty/saloweyca/clas%20395/pots/sld013.htm', 0.5150595512328365), ('http://www1.hollins.edu/docs/alumdev/confidinfovolun.htm', 0.4653336381891923), ('http://www1.hollins.edu/faculty/saloweyca/clas%20395/Acropolis/tsld010.htm', 0.38310315286418234), ('http://www1.hollins.edu/faculty/clarkjm/Stat140/140pic3.jpg', 0.1723986714033513), ('http://www1.hollins.edu/docs/academics/divisioni/classical%20studies/saloweyca/clas%20245/arapacis/apa.julioclaudian3.jpg', 0.161344060483984), ('http://www1.hollins.edu/Registrar/Forms/Intermaj.pdf', 0.15488538442921163), ('http://www1.hollins.edu/docs/Career_Development/majors/links/child.htm', 0.152821925892484), ('http://www1.hollins.edu/registrar/MAJORS-MINORS/MAJ-MIN%2099-00/COMPUTATIONALSCIENCES%20MATH-STAT%20CONC.pdf', 0.15254467377584097), ('http://www1.hollins.edu/homepages/saloweyca/

In [49]:
# Subset rankings for 25 iterations of page rank

# Sort all the subsets by their page rank
sorted_10 = [sorted(set, key=lambda url: get_pagerank(url)[2], reverse=True) for set in subsets_of_size_10]
sorted_50 = [sorted(set, key=lambda url: get_pagerank(url)[2], reverse=True) for set in subsets_of_size_50]

print("Size 10 subsets for 25 iterations of PageRank")
for set in sorted_10: 
    print([(url, get_pagerank(url)[2]) for url in set])

print("Size 50 subsets for 25 iterations of PageRank")
for set in sorted_50: 
    print([(url, get_pagerank(url)[2]) for url in set])

Size 10 subsets for 25 iterations of PageRank
[('http://www1.hollins.edu/faculty/saloweyca/clas%20395/pots/sld013.htm', 0.5411666840644911), ('http://www.hollins.edu/academics/library/information/gov_tax.htm', 0.5367816038157368), ('http://www1.hollins.edu/docs/alumdev/confidinfovolun.htm', 0.46593622349623576), ('http://www1.hollins.edu/faculty/saloweyca/clas%20395/Acropolis/tsld010.htm', 0.39185069031720277), ('http://www1.hollins.edu/faculty/clarkjm/Stat140/140pic3.jpg', 0.1723988964955177), ('http://www1.hollins.edu/docs/academics/divisioni/classical%20studies/saloweyca/clas%20245/arapacis/apa.julioclaudian3.jpg', 0.161344060483984), ('http://www1.hollins.edu/Registrar/Forms/Intermaj.pdf', 0.15489728881948178), ('http://www1.hollins.edu/docs/Career_Development/majors/links/child.htm', 0.15282199066442403), ('http://www1.hollins.edu/registrar/MAJORS-MINORS/MAJ-MIN%2099-00/COMPUTATIONALSCIENCES%20MATH-STAT%20CONC.pdf', 0.15254479798417545), ('http://www1.hollins.edu/homepages/salowey

It appears that the orderings change very minimally if at all as we adjusted the number of iterations for PageRank. This is an interesting observation since we observed significant changes in the the overall PageRanking ordering when we adjusted the iterations.

For the most part, it looks like the PageRanks of each webpage simply increased as the iterations of PageRank increased. So, we can observe a noticeable increase in PageRank score while the relative subset rankings stayed the same.